<h1 style="font-family:Didot; text-align:center;">RMSE dla modeli</h1>

In [78]:
import pandas as pd
import numpy as np
import os
from Arima import Arima
from boosting import XGBoost
from copy import deepcopy

In [28]:
def rmse(y, y_hat):
    return np.sqrt(np.sum((y - y_hat)**2) / len(y))

In [29]:
df = pd.read_csv('final_data.csv')
countries = df["Country"].unique()
continents = [df[df["Country"] == i]["Continent"].iloc[0] for i in countries]

In [73]:
new_df = pd.DataFrame({'Continent': continents, 'Country': countries})

In [93]:
def country_loss(model, country, years, *args):
    data = df[df["Country"] == country]["AverageTemperature"].to_numpy()
    training, test = data[:years*12], data[:-years*12]
    model_ = model(training, *args)
    model_.train()
    predictions = model_.predict(len(test))
    return rmse(test, predictions)


In [123]:
os.chdir(r"C:\\Users\\Mark Kostriew\\time_series_prediction")

In [121]:
def loss_to_files(model, model_name, *args): 
    os.chdir(os.path.dirname(os.path.realpath(__name__)))
    os.chdir(r"loss/test/country")
    df_ = deepcopy(new_df)
    vec = np.zeros(len(df_['country']))
    for i in range(len(countries)):
        vec[i] = country_loss(model, countries[i], 10, *args)
    df_['Loss'] = vec
    df_.to_csv(f'{model_name}.csv', index=False)  

In [118]:
loss_to_files(XGBoost, "XGBoost", 24)

In [122]:
os.chdir(r"C:\\Users\\Mark Kostriew\\time_series_prediction")
loss_to_files(Arima, "Arima")

C:\Users\Anaconda\envs\scientific_pyt\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Anaconda\envs\scientific_pyt\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Anaconda\envs\scientific_pyt\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Anaconda\envs\scientific_pyt\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Anaconda\envs\scientific_pyt\lib\site-p

In [124]:
continents_ = df["Continent"].unique()

In [126]:
continents_

array(['Asia', 'Europe', 'Africa', 'Oceania', 'North America',
       'South America'], dtype=object)

In [135]:
def continent_loss(model_name):
    df = pd.read_csv(rf"loss/test/country/{model_name}.csv")
    new_df = pd.DataFrame({'Continent': continents_})
    func = lambda x: df[df["continent"] == x]['loss'].mean()
    new_df['Loss'] = new_df['Continent'].apply(func)
    new_df.to_csv(rf"loss/test/continent/{model_name}.csv", index=False)

In [136]:
continent_loss("XGBoost")

In [137]:
continent_loss("Arima")

In [138]:
a = pd.read_csv(r"loss/test/country/Arima.csv")

In [141]:
a.columns = ['Continent', 'Country', 'Loss']

In [143]:
a.to_csv(r'loss/test/country/Arima.csv', index=False)

In [144]:
b = pd.read_csv(r"loss/test/country/XGBoost.csv") 

In [145]:
b.columns = ['Continent', 'Country', 'Loss']

In [147]:
b.to_csv(r'loss/test/country/XGBoost.csv', index=False)